# Small python notebook for retrieving user_ids for unlockable campaigns

In [57]:
import pandas as pd
from pandas import DataFrame, Series
import datetime
import numpy as np
import pyodbc
import os

In [58]:
conn = pyodbc.connect('DSN=Redshift_live')

conn.setencoding('utf-8')

def query(q):
    return pd.read_sql(q, conn)

In [59]:
import platform
import json
import datetime

def query(q):
    print("query started at: {0}".format(datetime.datetime.now().time()))
    df = pd.read_sql(q, conn, chunksize = None)
    print("query ended at: {0}".format(datetime.datetime.now().time()))    
    #if platform.system() == 'Windows':
    #    conn.close()
    return df

def list_to_string(lst):
    s = ",".join(map(str, lst))   
    return s

In [26]:
data_path = '/Users/gadir/Python/Unlockable/tables'



if not os.path.exists(data_path+'//data//'):
    os.makedirs(data_path+'//data//')

In [19]:
def get_unlock_user_ids(treatment_ids):
    treatment_ids = list_to_string(treatment_ids)
    q = """
     select 
         cte.user_id, 
         ctt.id as treatment_id,
         count(o.id) as rides_count
     from public.campaign_targeting_treatment ctt 
     left join public.campaign_targeting_enrollment cte on cte.treatment_id = ctt.id 
     left join company_order o on o.user_id = cte.user_id 
         and o.company_id IS null --historic dispatch button
            AND o.reseller_company_id IS NULL --historic dispatch button
            and o.state = 'finished' --finished from both rider and driver side
            and o.created::date > cte.created::date
     LEFT JOIN public.company_order_try ot ON ot.order_id=o.id 
         AND ot.city_id=o.city_id 
         and ot.retry_to_try IS null 
         and ot.state = 'finished' 
    where
    ctt.id in ({0})
    and cte.state = 'finished'
     group by 1,2
     --having rides_count > 1
    """.format(treatment_ids)
    
    return q

In [20]:
def get_rfm_user_ids(treatment_ids):
    treatment_ids = list_to_string(treatment_ids)
    q = """
     select cte.user_id
 from public.campaign_targeting_treatment ctt 
 left join public.campaign_targeting_enrollment cte on cte.treatment_id = ctt.id 
 where ctt.id in ({0})
    """.format(treatment_ids)
    
    return q

In [21]:
#get info about campaigns 

info = query('''
select 
	t.name test_name,
	tt.id treatment_id,
	tt.bonus_type,
	case when tt.bonus_data like '%targeted_campaign_id%' 
        then json_extract_path_text(tt.bonus_data,'targeted_campaign_id')::int else null end as targeted_campaign_id,
	c.name campaign_name,
	case when c.bonus_data like '%percentage%' 
        then  json_extract_path_text(c.bonus_data,'percentage') else '?' end as  campaign_percentage,
	case when c.bonus_data like '%max_value%' 
        then  json_extract_path_text(c.bonus_data,'max_value')  else '?' end as campaign_max_value,
	count(e.user_id) enrollment_count
from campaign_targeting_enrollment e 
left join campaign_targeting_test t on t.id = e.test_id
left join campaign_targeting_treatment tt on tt.id = e.treatment_id
left join campaign_targeted_campaign c on c.id = case when tt.bonus_data like '%targeted_campaign_id%' then json_extract_path_text(tt.bonus_data,'targeted_campaign_id')::int else null end
where 
	t.city_id = (select id from admin_system_city where name = 'Durban')
	and t.created::date > '2021-01-20'
	and tt.distribution = 5
	and e.state = 'finished'
group by 1,2,3,4,5,6,7
''')


query started at: 20:23:48.873066
query ended at: 20:23:56.688201


In [60]:
info.sort_values('campaign_name')

,test_name,treatment_id,bonus_type,targeted_campaign_id,campaign_name,campaign_percentage,campaign_max_value,enrollment_count
6,"ZA, Durban, 2021W4, Low activity",34911,targeted_campaign,22992.0,"ZA, Durban, 2021W4, LA, -20% x 10 (10 ZAR) C",20,10,4249
10,"ZA, Durban, 2021W4, Low activity",34912,targeted_campaign,22994.0,"ZA, Durban, 2021W4, LA, -40% x 10 (19 ZAR) D",40,19,4208
12,"ZA, Durban, 2021W4, Not active",34907,targeted_campaign,22991.0,"ZA, Durban, 2021W4, NA, -30% x 10 (15 ZAR) C",30,15,32688
24,"ZA, Durban, 2021W4, Not active",34908,targeted_campaign,22993.0,"ZA, Durban, 2021W4, NA, -50% x 10 (24 ZAR) D",50,24,33234
25,"ZA, Durban, 2021W4, unlockable lock test, HR",34735,targeted_campaign,22904.0,"ZA, Durban, 2021W4, unlockable lock test, -10%...",10,5,4772
15,"ZA, Durban, 2021W4, unlockable lock test, MA",34727,targeted_campaign,22904.0,"ZA, Durban, 2021W4, unlockable lock test, -10%...",10,5,2165
3,"ZA, Durban, 2021W4, unlockable lock test, HF",34743,targeted_campaign,22904.0,"ZA, Durban, 2021W4, unlockable lock test, -10%...",10,5,4655
1,"ZA, Durban, 2021W4, unlockable lock test, MA",34728,targeted_campaign,22905.0,"ZA, Durban, 2021W4, unlockable lock test, -20%...",20,10,2144
4,"ZA, Durban, 2021W4, unlockable lock test, HR",34736,targeted_campaign,22905.0,"ZA, Durban, 2021W4, unlockable lock test, -20%...",20,10,4903
2,"ZA, Durban, 2021W4, unlockable lock test, HF",34744,targeted_campaign,22905.0,"ZA, Durban, 2021W4, unlockable lock test, -20%...",20,10,4741


In [23]:
info.fillna('na').groupby(["bonus_type","campaign_name","treatment_id"])["enrollment_count"].sum()

bonus_type                 campaign_name                                                 treatment_id
targeted_campaign          ZA, Durban, 2021W4, LA, -20% x 10 (10 ZAR) C                  34911            4249
                           ZA, Durban, 2021W4, LA, -40% x 10 (19 ZAR) D                  34912            4208
                           ZA, Durban, 2021W4, NA, -30% x 10 (15 ZAR) C                  34907           32688
                           ZA, Durban, 2021W4, NA, -50% x 10 (24 ZAR) D                  34908           33234
                           ZA, Durban, 2021W4, unlockable lock test, -10% x 10 (5 ZAR)   34727            2165
                                                                                         34735            4772
                                                                                         34743            4655
                           ZA, Durban, 2021W4, unlockable lock test, -20% x 10 (10 ZAR)  34728            2144
          

## Get users from unlockable campaigns 
### who finished 2 or more rides

In [61]:
#1st week - 10%, 2nd week - 20%
treatment_ids = [34727,34735,34743]      

In [62]:
df = query(get_unlock_user_ids(treatment_ids))

query started at: 05:14:24.809565
query ended at: 05:14:27.713102


In [63]:
print(len(df))
print(len(df)/(info[info["treatment_id"].isin(treatment_ids)].enrollment_count.sum()))
print(len(df[df["rides_count"]>1])/(info[info["treatment_id"].isin(treatment_ids)].enrollment_count.sum()))
a = df.groupby("rides_count")["user_id"].count()
a
df[df['rides_count']>1].count()

11592
1.0
0.05684955141476881


user_id         659
treatment_id    659
rides_count     659
dtype: int64

In [57]:
#save only user_ids to csv 
df[df["rides_count"]>1].user_id.to_csv('unlockable_20.csv', index = False, header = False)

In [64]:
#save string file 

string = list_to_string(df[df["rides_count"]>1].user_id)
print(string,  file=open('unlockable_20.txt', 'w'))

In [72]:
#1st week - 20%, 2nd week - 30%
treatment_ids = [34728,34736,34744]

In [73]:
df = query(get_unlock_user_ids(treatment_ids))
df.head()

query started at: 05:17:23.335103
query ended at: 05:17:31.530586


,user_id,treatment_id,rides_count
0,47709268,34728,2
1,29709253,34736,0
2,48978623,34736,0
3,26278412,34744,0
4,7047408,34736,0


In [74]:
print(len(df))
print(len(df)/(info[info["treatment_id"].isin(treatment_ids)].enrollment_count.sum()))
print(len(df[df["rides_count"]>1])/(info[info["treatment_id"].isin(treatment_ids)].enrollment_count.sum()))
a = df.groupby("rides_count")["user_id"].count()
a
df[df['rides_count']>1].count()

11788
1.0
0.07244655581947744


user_id         854
treatment_id    854
rides_count     854
dtype: int64

In [28]:
#save only user_ids to csv 
df[df["rides_count"]>1].user_id.to_csv('unlockable_30.txt', index = False, header = False)

In [75]:
#save string file 

string = list_to_string(df[df["rides_count"]>1].user_id)
print(string,  file=open('unlockable_30.txt', 'w'))

In [68]:
#1st week - 30%, 2nd week - 40%
treatment_ids = [34729,34737,34745]

In [69]:
df = query(get_unlock_user_ids(treatment_ids))
df.head()

query started at: 05:16:16.809807
query ended at: 05:16:42.521362


,user_id,treatment_id,rides_count
0,58193763,34737,3
1,47246972,34745,0
2,46298885,34737,0
3,42282560,34745,0
4,52607028,34745,0


In [70]:
print(len(df))
print(len(df)/(info[info["treatment_id"].isin(treatment_ids)].enrollment_count.sum()))
print(len(df[df["rides_count"]>1])/(info[info["treatment_id"].isin(treatment_ids)].enrollment_count.sum()))
a = df.groupby("rides_count")["user_id"].count()
a
df[df['rides_count']>1].count()

11675
1.0
0.07888650963597431


user_id         921
treatment_id    921
rides_count     921
dtype: int64

In [32]:
#save only user_ids to csv 
df[df["rides_count"]>1].user_id.to_csv('unlockable_40.txt', index = False, header = False)

In [71]:
#save string file 

string = list_to_string(df[df["rides_count"]>1].user_id)
print(string,  file=open('unlockable_40.txt', 'w'))

## Get users from RFM campaigns

In [27]:
#1st week - 10%, 2nd week - 10%
treatment_ids = [34730,34738,34746]

In [28]:
df = query(get_unlock_user_ids(treatment_ids))
#print(len(df))
#df.head()

query started at: 20:35:24.344275
query ended at: 20:35:39.092413


In [29]:
print(len(df))
print(len(df)/(info[info["treatment_id"].isin(treatment_ids)].enrollment_count.sum()))
print(len(df[df["rides_count"]>1])/(info[info["treatment_id"].isin(treatment_ids)].enrollment_count.sum()))
a = df.groupby("rides_count")["user_id"].count()
a
df['rides_count'].count()

11538
1.0
0.056335586756803606


11538

In [36]:
df.user_id.to_csv('rfm_unlock_20.txt', index = False, header = False)

In [30]:
#save string file 

string = list_to_string(df.user_id)
print(string,  file=open('rfm_unlock_20.txt', 'w'))

In [31]:
#1st week - 20%, 2nd week - 20%
treatment_ids = [37431, 34739, 34747]

In [32]:
df = query(get_unlock_user_ids(treatment_ids))
#print(len(df))
#df.head()

query started at: 20:41:32.294638
query ended at: 20:41:35.257841


In [33]:
print(len(df))
print(len(df)/(info[info["treatment_id"].isin(treatment_ids)].enrollment_count.sum()))
print(len(df[df["rides_count"]>1])/(info[info["treatment_id"].isin(treatment_ids)].enrollment_count.sum()))
a = df.groupby("rides_count")["user_id"].count()
a
df.user_id.count()

9460
1.0
0.06775898520084567


9460

In [40]:
df.user_id.to_csv('rfm_unlock_30.txt', index = False, header = False)

In [35]:
#save string file 

string = list_to_string(df.user_id)
print(string,  file=open('rfm_unlock_30.txt', 'w'))

In [36]:
#1st week - 30%, 2nd week - 30%
treatment_ids = [34732, 34740, 34748]

In [37]:
df = query(get_unlock_user_ids(treatment_ids))
#print(len(df))
#df.head()

query started at: 20:42:35.992589
query ended at: 20:42:38.241930


In [38]:
print(len(df))
print(len(df)/(info[info["treatment_id"].isin(treatment_ids)].enrollment_count.sum()))
print(len(df[df["rides_count"]>1])/(info[info["treatment_id"].isin(treatment_ids)].enrollment_count.sum()))
a = df.groupby("rides_count")["user_id"].count()
a
df['rides_count'].count()

11712
1.0
0.0706967213114754


11712

In [44]:
df.user_id.to_csv('rfm_unlock_40.txt', index = False, header = False)

In [39]:
#save string file 

string = list_to_string(df.user_id)
print(string,  file=open('rfm_unlock_40.txt', 'w'))

# Overview

In [49]:
treatment_ids = info.treatment_id.unique()


In [50]:
df = query(get_unlock_user_ids(treatment_ids))
#print(len(df))

query started at: 20:44:13.441531
query ended at: 20:44:30.189197


In [51]:
info.head()

,test_name,treatment_id,bonus_type,targeted_campaign_id,campaign_name,campaign_percentage,campaign_max_value,enrollment_count
0,"ZA, Durban, 2021W4, Not active",34909,targeted_control_campaign,NaN,None,?,?,32677
1,"ZA, Durban, 2021W4, unlockable lock test, MA",34728,targeted_campaign,22905.0,"ZA, Durban, 2021W4, unlockable lock test, -20%...",20,10,2144
2,"ZA, Durban, 2021W4, unlockable lock test, HF",34744,targeted_campaign,22905.0,"ZA, Durban, 2021W4, unlockable lock test, -20%...",20,10,4741
3,"ZA, Durban, 2021W4, unlockable lock test, HF",34743,targeted_campaign,22904.0,"ZA, Durban, 2021W4, unlockable lock test, -10%...",10,5,4655
4,"ZA, Durban, 2021W4, unlockable lock test, HR",34736,targeted_campaign,22905.0,"ZA, Durban, 2021W4, unlockable lock test, -20%...",20,10,4903


In [52]:
df = df.merge(info[["treatment_id","campaign_name"]], on = "treatment_id", how = "left" )

In [53]:
print(len(df))
print(len(df)/(info[info["treatment_id"].isin(treatment_ids)].enrollment_count.sum()))
print(len(df[df["rides_count"]>1])/(info[info["treatment_id"].isin(treatment_ids)].enrollment_count.sum()))
a = df.groupby("rides_count")["user_id"].count()
a

188645
1.0
0.029457446526544567


rides_count
0     174903
1       8185
2       3237
3       1166
4        556
5        274
6        127
7         75
8         51
9         26
10        23
11         9
12         5
13         5
14         2
15         1
Name: user_id, dtype: int64

In [54]:
df["had_1+ride"]=np.where(df.rides_count>0,1,0)
df["had_2+ride"]=np.where(df.rides_count>1,1,0)

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide();
    } else {
        $('div.input').show();
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Toggle code view"></form>''')


In [55]:
a = pd.pivot_table(df,index = "campaign_name", columns = "had_1+ride", values = "user_id", aggfunc = "count")#.reset_index()
a["1+_share"] = a[1]/(a[0]+a[1])
a

had_1+ride,0,1,1+_share
campaign_name,,,
"ZA, Durban, 2021W4, LA, -20% x 10 (10 ZAR) C",4071,178,0.041892
"ZA, Durban, 2021W4, LA, -40% x 10 (19 ZAR) D",4024,184,0.043726
"ZA, Durban, 2021W4, NA, -30% x 10 (15 ZAR) C",32486,202,0.006180
"ZA, Durban, 2021W4, NA, -50% x 10 (24 ZAR) D",33003,231,0.006951
"ZA, Durban, 2021W4, unlockable lock test, -10% x 10 (5 ZAR)",9857,1735,0.149672
"ZA, Durban, 2021W4, unlockable lock test, -20% x 10 (10 ZAR)",9859,1929,0.163641
"ZA, Durban, 2021W4, unlockable lock test, -30% x 10 (15 ZAR)",9662,2013,0.172420
"ZA, Durban, 2021W4, unlockable rfm test, -10% x 10 (5 ZAR)",9814,1724,0.149419
"ZA, Durban, 2021W4, unlockable rfm test, -20% x 10 (10 ZAR)",9769,1869,0.160595


In [56]:
a = pd.pivot_table(df,index = "campaign_name", columns = "had_2+ride", values = "user_id", aggfunc = "count")#.reset_index()
a["2+_share"] = a[1]/(a[0]+a[1])
a

had_2+ride,0,1,2+_share
campaign_name,,,
"ZA, Durban, 2021W4, LA, -20% x 10 (10 ZAR) C",4193,56,0.013180
"ZA, Durban, 2021W4, LA, -40% x 10 (19 ZAR) D",4150,58,0.013783
"ZA, Durban, 2021W4, NA, -30% x 10 (15 ZAR) C",32616,72,0.002203
"ZA, Durban, 2021W4, NA, -50% x 10 (24 ZAR) D",33161,73,0.002197
"ZA, Durban, 2021W4, unlockable lock test, -10% x 10 (5 ZAR)",10941,651,0.056159
"ZA, Durban, 2021W4, unlockable lock test, -20% x 10 (10 ZAR)",10941,847,0.071853
"ZA, Durban, 2021W4, unlockable lock test, -30% x 10 (15 ZAR)",10766,909,0.077859
"ZA, Durban, 2021W4, unlockable rfm test, -10% x 10 (5 ZAR)",10888,650,0.056336
"ZA, Durban, 2021W4, unlockable rfm test, -20% x 10 (10 ZAR)",10882,756,0.064960


In [16]:
import pandas as pd 
need = pd.read_csv('cpt_prem.csv')
string = list_to_string(need.user_id)
print(string,  file=open('cpt_prem.txt', 'w'))

In [11]:
from sklearn.model_selection import train_test_split
import numpy

# with open("datafile.txt", "rb") as f:
#    data = f.read().split('\n')
#    data = numpy.array(data)  #convert array to numpy type array

data = [1,3,4,5,6345,54,645,77,65,79,83,995]
x_train ,x_test = train_test_split(data,test_size=0.5)

In [12]:
x_train

[995, 1, 4, 3, 54, 65]

In [14]:
x_test

[83, 79, 5, 645, 77, 6345]